## Step 1. Load the Data

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

file_path = r'C:\Users\user\Downloads\2024 솔챌\유투브영상수집.xlsx'
df = pd.read_excel(file_path)


In [2]:
#pip install openpyxl

## Step 2. Define Input Processing

In [3]:
# List of all emotion and event columns in the dataset
emotion_columns = ['angry', 'sad', 'anxious', 'hurt', 'embarrassed', 'happy']
event_columns = ['relationship', 'breakup', 'family', 'work', 'studying', 'insomnia', 'people', 'health', 'self-esteem', 'money', 'friends']

# Function to concatenate applicable tags for each video
def create_tags(row):
    tags = []
    for col in emotion_columns + event_columns:
        if row[col] == 1:  # Assuming 1 indicates an applicable tag
            tags.append(col)
    return ' '.join(tags)  # Concatenating all tags into a single string

# Applying the function to each row in the dataframe to create the 'tags' column
df['tags'] = df.apply(create_tags, axis=1)

In [4]:
# vectorize 방식 변경 
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['tags'])

## Step 3. Implement the Recommendation Logic

In [5]:
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(X)
df['cluster'] = kmeans.labels_

c:\Users\user\.conda\envs\AI_2023\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\user\.conda\envs\AI_2023\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 지정된 파일을 찾을 수 없습니다
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\user\.conda\envs\AI_2023\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\user\.conda\envs\AI_2023\lib\subprocess.py", line 505, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\user\.conda\envs\AI_2023\li

In [6]:
def recommend_clusters(input_tags, vectorizer, kmeans):
    input_vec = vectorizer.transform([" ".join(input_tags)])
    similarities = cosine_similarity(input_vec, kmeans.cluster_centers_)
    scores = similarities.flatten()
    scores_dict = {i: scores[i] for i in range(len(scores))}
    return scores_dict

In [7]:
def recommend_videos(df, cluster_scores, kmeans, vectorizer):
    recommend = []
    for cluster_index in sorted(cluster_scores, key=cluster_scores.get, reverse=True):
        cluster_videos = df[df['cluster'] == cluster_index]
        distances = cosine_similarity(vectorizer.transform(cluster_videos['tags']), kmeans.cluster_centers_[cluster_index].reshape(1, -1))
        cluster_videos['distance'] = distances.flatten()
        sorted_videos = cluster_videos.sort_values(by='distance', ascending=False)
        recommend.extend(sorted_videos['링크'].tolist())
    return recommend

## Step 4. Apply the Algorithm

In [8]:
input_tags = ['breakup']  # 사용자 입력
cluster_scores = recommend_clusters(input_tags, vectorizer, kmeans)
recommended_videos = recommend_videos(df, cluster_scores, kmeans, vectorizer)

for url in recommended_videos[:5]:
    print(url)

https://youtu.be/SELP_6uBy7I?feature=shared
https://youtu.be/Odok80rhavI?feature=shared
https://youtu.be/vnTnmWJcbXo?feature=shared
https://youtu.be/tIVtOGmpSsk?feature=shared
https://youtu.be/R7nd1CtDWBU?feature=shared


C:\Users\user\AppData\Local\Temp\ipykernel_12628\2327298254.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_videos['distance'] = distances.flatten()
C:\Users\user\AppData\Local\Temp\ipykernel_12628\2327298254.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_videos['distance'] = distances.flatten()
C:\Users\user\AppData\Local\Temp\ipykernel_12628\2327298254.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i